# Mocking internals of a Python Script
> How to test against a Python script when you need to change how `__main__` behaves

This will be split into two parts. The first contains the contents of some `script.py` file, which is the base template script we want to use:

In [ ]:
def test_function(): 
    return 2

def main():
    result = test_function()
    with open("someFile.txt", "w") as f:
        f.write(str(result))
        
if __name__ == "__main__":
    main()

#| tip "def" 0 "return 2" 0

#| filename: examples/script.py

This is the function whose behavior we want to override during our mock

#| tip "def" 1 "))" 

The main function is what we will call when running the python script. This will write a string representation of `test_function()` to a file.

#| tip "if" 0 "()" 3
This is a blocker to ensure that if anyone imports or calls this python script that it will be ran explicitly.

::: {.callout-warning}

## File Structure

It should be assumed that for the next part the structure of the code files are as such:

* `base_repository`
  * `example`
    * `script.py`
  * `tests`
    * `test_script.py`
:::

In [ ]:
import os
import sys
import unittest
from unittest import mock
SRC_DIRS = [
    os.path.join(
        os.path.dirname(__file__), "example"
    )
]
sys.path.extend(SRC_DIRS)
if SRC_DIRS is not None:
    import script
def new_function():
    return 0
@mock.patch("script.test_function", new_function)
class ExampleTester(unittest.TestCase):
    def test_example(self):
        script.main()
        with open("someFile.txt", "r") as f:
            lines = f.read()
        self.assertEquals(lines, "0")

#| tip "SRC_DIRS" 0 ] 0
This is a list of directories that have our script source code relative to the current file. In this case the `example` directory.

#| tip "sys.path" 0 "SRC_DIRS)"

We add in our new SRC_DIRS to the sys.path which allows them to be imported through an import statement such as import script

#| tip "if SRC_DIRS" 0 "import script"

If the file exists (this makes it modular) go ahead and import it

#| tip "def new_" 0 "return 0" 0
This is the new function we will use to replace the `test_function` in our python script

#| tip "@mock" 0 "new_function)"

This uses `unittest.mock` to mokey-patch and override the original `test_function` in the existing module with the new one we just defined. Calling `script.test_function()` will call `new_function()` as a result

#| tip "class" 0 ) 7

Calls the main function in our tester, but uses our `new_function()` when called

#| tip "with open" 0 ")" 10

Tests that the file which was written to has the properly mocked version of it, or 0